In [1]:
# Loading the data
import pandas as pd
import numpy as np
titanic_train = pd.read_csv('/kaggle/input/titanic/train.csv')
titanic_test = pd.read_csv('/kaggle/input/titanic/test.csv')
titanic_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
# Handling the missing values 
print('Missing Values in Training Data')
print(titanic_train.isnull().sum())

print('Missing Values in Test Data:')
print(titanic_test.isnull().sum())
# Fix missing values
titanic_train.drop('Cabin', axis=1, inplace=True)  # Drop 'Cabin' since it's mostly empty
titanic_train['Embarked'].fillna(titanic_train['Embarked'].mode()[0], inplace=True)  # Fill 'Embarked' with mode
titanic_train['Age'].fillna(titanic_train['Age'].mean(), inplace=True)  # Fill 'Age' with mean

# Fix missing 'Fare' in test data
titanic_test['Fare'].fillna(titanic_test['Fare'].median(), inplace=True)
titanic_test['Age'].fillna(titanic_test['Age'].mean(), inplace=True)

Missing Values in Training Data
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
Missing Values in Test Data:
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


<ipython-input-2-2b538c42eb98>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  titanic_train['Embarked'].fillna(titanic_train['Embarked'].mode()[0], inplace=True)  # Fill 'Embarked' with mode
<ipython-input-2-2b538c42eb98>:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exam

In [3]:
# Converting 'Sex' in to numerical values male =0 , female = 1
titanic_train['Sex'] = titanic_train['Sex'].map({'male':0, 'female':1})
titanic_test['Sex'] = titanic_test['Sex'].map({'male':0, 'female':1})
# Converting Embarked Vlues to numericl values C=0,Q=1, S=2
titanic_train['Embarked'] = titanic_train['Embarked'].map({'C':0,'Q':1,'S':2})
titanic_test['Embarked'] = titanic_test['Embarked'].map({'C':0,'Q':1,'S':2})
titanic_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,2
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,2


In [4]:
# Target value
y = titanic_train['Survived']
# Select features for training 
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = titanic_train[features]
X.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,2.308642,0.352413,29.699118,0.523008,0.381594,32.204208,1.536476
std,0.836071,0.477990,13.002015,1.102743,0.806057,49.693429,0.791503
min,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,22.000000,0.000000,0.000000,7.910400,1.000000
50%,3.000000,0.000000,29.699118,0.000000,0.000000,14.454200,2.000000
75%,3.000000,1.000000,35.000000,1.000000,0.000000,31.000000,2.000000
max,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200,2.000000


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
# Splitting the data into Training and Validation
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
# Initalizing RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(train_X, train_y)

RandomForestClassifier(max_depth=5, random_state=1)

In [6]:
# Validating model 
val_predictions = model.predict(val_X)
val_mae = mean_absolute_error(val_predictions,val_y)
val_accuracy = accuracy_score(val_predictions,val_y)
print('Validation Accuracy:', val_accuracy)
print('Validation MAE:', val_mae)

Validation Accuracy: 0.7802690582959642
Validation MAE: 0.21973094170403587


In [7]:
# Training the model on enitire Data 
from sklearn.ensemble import RandomForestClassifier
final_model=RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
final_model.fit(X, y)

RandomForestClassifier(max_depth=5, random_state=1)

In [8]:
#  Make predictions on the test dataset
test_X = titanic_test[features]
test_predictions = final_model.predict(test_X)

In [9]:
# Saving predictions to CSV for submission
submission = pd.DataFrame({'PassengerId': titanic_test['PassengerId'], 'Survived': test_predictions})
submission.to_csv('submission.csv', index=False)

print("Submission file saved as 'submission.csv'")

Submission file saved as 'submission.csv'
